# **PHASE 3: ADVANCED ANALYTICS (Days 9-11)**

## **DAY 10 (18/01/26) - Performance Optimization**



### **Section 1 - Learn**:

### **_1. SQL Warehouse_**

Understanding **Query Execution Plans** is essential for debugging slow Spark or SQL code. In Databricks, the Execution Plan is the "blueprint" Spark creates to decide how to read, join, and aggregate your data across the cluster.

##### **1. The Spark Optimizer (Catalyst)**

When you run a query, Spark doesn't execute it immediately. It passes your code through the **Catalyst Optimizer**, which transforms it through four stages:

1. **Analysis:** Checks the SQL/Code for syntax errors and ensures tables/columns exist in the Catalog.
2. **Logical Optimization:** Applies rule-based optimizations, like "Pushdown Filters" (filtering data as early as possible so less data is moved).
3. **Physical Planning:** Generates multiple physical strategies (e.g., deciding between a Broadcast Join vs. a Shuffle Hash Join) and picks the one with the lowest "cost."
4. **Code Generation (Tungsten):** Generates highly optimized Java bytecode to run on the workers.

##### **2. How to View the Plan**

* **SQL:** Use the command `EXPLAIN EXTENDED SELECT ...`
* **PySpark:** Use `df.explain(True)`
* **Spark UI:** The "SQL" tab in the Spark UI provides a visual DAG (Directed Acyclic Graph) of the execution, which is the most user-friendly way to spot bottlenecks.

##### **3. Key Terms to Look For**

When reading a plan, you want to identify these specific operations:

| Operation | What it means | Performance Impact |
| --- | --- | --- |
| **FileScan** | Reading data from storage (S3/ADLS). | Check "PartitionFilters" and "PushedFilters" to see if data skipping is working. |
| **Exchange** | A **Shuffle**. Data is being moved across the network between nodes. | **Expensive.** High exchange usually means a large join or a wide group-by. |
| **BroadcastExchange** | Sending a small table to all nodes. | **Fast.** Much better than a full shuffle for joins. |
| **Project** | Selecting specific columns. | Low cost. |
| **Filter** | Filtering rows. | Low cost; highly efficient if pushed down to the data source. |

##### **4. Red Flags in a Query Plan**

* **CartesianProduct:** This happens if you join two tables without a join key. It causes an exponential explosion of data and usually crashes the cluster.
* **ObjectHashAggregate:** Usually faster than `SortAggregate`, but if you see `Sort`, it means Spark had to sort the data on disk first, which is slower.
* **Scan with no Filters:** If you are querying a massive table but the plan shows no `PartitionFilters`, Spark is doing a "Full Table Scan," which is extremely inefficient.

##### **5. Performance Tuning via the Plan**

If you see a massive **Exchange** node (Shuffle) that is slowing you down:

1. **Check Join Types:** Can you use a `broadcast()` hint for the smaller table?
2. **Z-Order:** If the `FileScan` is taking too long, does the table need to be Z-Ordered on the filter column?
3. **Data Skew:** If one task in the Spark UI takes 10 minutes while others take 10 seconds, you have "Data Skew." You may need to use a `SKEW` hint or salt your keys.

---

### **_2. Partitioning strategies_**

In Databricks, partitioning is the process of physically grouping data into folders based on a specific column (like `/year=2026/month=01/`). While it was once the standard for all big data tables, modern best practices have shifted toward more automated solutions.

##### **1. The Golden Rule: 1 TB**

* **Small Tables (< 1 TB):** Do **not** partition. The overhead of managing separate folders and files actually slows down queries. Spark's metadata handling is efficient enough to scan 1 TB without partitioning.
* **Large Tables (> 1 TB):** Partitioning becomes beneficial because it allows for **Partition Pruning**, where Spark skips entire directories of data that don't match your query.

##### **2. Choosing the Right Column**

Choosing the wrong column can lead to the **"Small File Problem"** or unbalanced data.

* **Low Cardinality:** Choose columns with a limited number of unique values (e.g., `Date`, `Region`, `Department`).
* **Avoid High Cardinality:** Never partition by `User_ID`, `Transaction_ID`, or `Timestamp`. If a column has more than 10,000 unique values, partitioning will create too many tiny folders, crushing performance.
* **Uniformity:** Ensure your partitions are roughly the same size. If 90% of your data is in one "Region," partitioning by region won't help much.

##### **3. Liquid Clustering: The Modern Alternative**

In 2026, Databricks recommends **Liquid Clustering** over traditional partitioning for almost all new tables.

| Feature | Traditional Partitioning | Liquid Clustering |
| --- | --- | --- |
| **Setup** | Fixed at table creation. | Flexible; can be changed anytime. |
| **Maintenance** | Manual (needs `ZORDER` frequently). | Automatic (predictive optimization). |
| **Cardinality** | Only for low cardinality. | Works for high and low cardinality. |
| **Data Skew** | Vulnerable to uneven partitions. | Naturally handles data skew. |

**Best Practice:** Use Liquid Clustering (`CLUSTER BY`) unless you have a legacy requirement for specific folder structures or a very static, massive table (100 TB+) with a single, predictable filter.

##### **4. Implementation Example**

**Traditional Partitioning (Legacy/Static):**

```python
df.write \
  .format("delta") \
  .partitionBy("event_date") \
  .saveAsTable("events_partitioned")

```

**Liquid Clustering (Recommended):**

```sql
-- Modern SQL approach
CREATE TABLE events_clustered
USING DELTA
CLUSTER BY (customer_id, event_date);

```

##### **Summary Table**

| Strategy | When to Use |
| --- | --- |
| **No Partitioning** | Tables under 1 TB. |
| **Partitioning** | Large tables (> 1 TB) with fixed, low-cardinality filters. |
| **Liquid Clustering** | **Default for 2026.** Evolving schemas, high cardinality, or skewed data. |


---

### **_3. OPTIMIZE & ZORDER_**

In Delta Lake, `OPTIMIZE` and `ZORDER` are the two primary tools used to physically reorganize data on disk to improve query performance and reduce storage overhead.

##### **1. OPTIMIZE (File Compaction)**

* **The "Small File Problem":** Frequent streaming or small batch writes often create thousands of tiny Parquet files. This slows down queries because the engine spends more time opening/closing files and reading metadata than actually processing data.
* **Bin-Packing:** The `OPTIMIZE` command performs "compaction." It takes those small files and merges them into larger, right-sized files (defaulting to **1GB**).
* **Metadata Efficiency:** By reducing the number of files, Spark has much less work to do when scanning the transaction log and listing files in cloud storage.

##### **2. ZORDER (Data Clustering)**

* **Multi-Dimensional Sorting:** While `OPTIMIZE` just makes files bigger, adding `ZORDER BY` physically **rearranges the rows** within those files based on specific columns.
* **Data Skipping:** It co-locates related information. If you Z-Order by `customer_id`, rows for the same customer will be grouped into the same file. When you filter for that ID, Delta Lake skips the files that don't contain it by checking the min/max statistics in the log.
* **High Cardinality Columns:** Z-Ordering is most effective on columns that have many unique values (like `ID`, `email`, or `phone_number`) and are frequently used in `WHERE` clauses or as `JOIN` keys.
* **The 4-Column Limit:** Effectiveness drops if you Z-Order by too many columns (usually stay under 1–4). It is a "trade-off" tool—the more columns you Z-Order, the less "tight" the clustering becomes for each one.

##### **Summary Comparison**

| Feature | OPTIMIZE | ZORDER |
| --- | --- | --- |
| **Primary Goal** | Solve the "Small File Problem" | Enable advanced "Data Skipping" |
| **What it does** | Merges small files into ~1GB files | Clusters related row values together |
| **When to run** | After many small writes/updates | On large tables with frequent filters |
| **Resource Cost** | Low to Moderate (I/O heavy) | High (Compute intensive sorting) |


##### **PySpark & SQL Usage**

```sql
-- SQL Syntax
OPTIMIZE my_table 
ZORDER BY (customer_id, event_type);

```

```python
# PySpark Syntax
from delta.tables import DeltaTable
deltaTable = DeltaTable.forPath(spark, "/path/to/table")
deltaTable.optimize().zorderBy("customer_id").executeZOrderBy()

```

---

### **_4. Caching techniques_**
In Databricks, caching is not a single "on/off" switch. Instead, there are several distinct layers of caching that serve different purposes. Understanding which one to use can save you significant time and compute costs.

##### **1. Disk Cache (Formerly Delta Cache)**

This is the most important cache for Databricks performance. It stores copies of remote Parquet/Delta files on the **local SSDs** of your worker nodes.

* **How it works:** It automatically caches data as it is read from cloud storage. Subsequent reads of the same data are fetched from the local SSD instead of the network.
* **Best Practice:** Choose a **"Delta Cache Accelerated"** worker type (like the `L-series` in Azure or `i-series` in AWS). These come with pre-configured SSDs specifically for this purpose.
* **Manual Control:** You can force a cache warm-up using `CACHE SELECT * FROM table_name`.

##### **2. Spark Cache (`.cache()` and `.persist()`)**

This is the traditional Spark mechanism that stores data in **JVM Memory (RAM)**.

* **Difference:** Unlike the Disk Cache (which stores raw file data), Spark Cache stores **computed DataFrames**. It is useful if you have a massive transformation (like a 10-way join) that you need to reuse multiple times in the same notebook.
* **Storage Levels:** `.cache()` uses `MEMORY_AND_DISK` by default. `.persist()` allows you to choose `DISK_ONLY` or `MEMORY_ONLY_SER` (serialized) to save space.
* **Warning:** Modern Databricks best practices suggest **avoiding Spark Caching** unless you are reusing an expensive intermediate result. The Disk Cache is usually faster and doesn't steal RAM from your Spark executors.

##### **3. Query Result Cache**

This is specific to **Databricks SQL Warehouses**.

* **How it works:** It stores the final results of a SQL query. If you (or anyone else in your workspace) run the exact same query again and the underlying data hasn't changed, Databricks returns the result **instantly** without spinning up the cluster.
* **Remote Result Cache:** In Serverless SQL, this cache is persistent across warehouse restarts, meaning you don't lose the "warm" results even if the warehouse shuts down at night.

##### **Summary Comparison Table**

| Feature | Disk Cache (Delta) | Spark Cache (`.cache`) | Result Cache |
| --- | --- | --- | --- |
| **Storage Medium** | Local SSD | RAM / JVM Memory | In-memory & Remote |
| **Stores...** | Raw file data (Parquet) | Transformed DataFrames | Final Query Results |
| **Trigger** | Automatic (on read) | Manual (`.cache()`) | Automatic (on SQL query) |
| **Best For...** | General query speedup | Iterative ML / Multi-use DFs | BI Dashboards |

##### **Performance Tip: Cache Invalidation**

You don't need to manually clear the Disk Cache. Databricks **automatically detects** if the underlying Delta table has been updated and will evict the old files. For the Spark Cache, however, you should always call `.unpersist()` when you are finished to free up RAM.

---

### **Practice**

#### ***1. Analyze query plans***

In [0]:
# Explain query
spark.sql("SELECT * FROM workspace.silver.ecommerce_cleaned WHERE event_type='purchase'").explain(True)

== Parsed Logical Plan ==
'Project [*]
+- 'Filter ('event_type = purchase)
   +- 'UnresolvedRelation [workspace, silver, ecommerce_cleaned], [], false

== Analyzed Logical Plan ==
event_time: timestamp, event_type: string, product_id: int, category_id: bigint, category_code: string, brand: string, price: double, user_id: int, user_session: string, ingestion_ts: timestamp, event_date: date, price_tier: string
Project [event_time#13527, event_type#13528, product_id#13529, category_id#13530L, category_code#13531, brand#13532, price#13533, user_id#13534, user_session#13535, ingestion_ts#13536, event_date#13537, price_tier#13538]
+- Filter (event_type#13528 = purchase)
   +- SubqueryAlias workspace.silver.ecommerce_cleaned
      +- Relation workspace.silver.ecommerce_cleaned[event_time#13527,event_type#13528,product_id#13529,category_id#13530L,category_code#13531,brand#13532,price#13533,user_id#13534,user_session#13535,ingestion_ts#13536,event_date#13537,price_tier#13538] parquet

== Optimi

In [0]:
%sql
explain extended select * from workspace.silver.ecommerce_cleaned 

plan
"== Parsed Logical Plan == 'Project [*] +- 'UnresolvedRelation [workspace, silver, ecommerce_cleaned], [], false == Analyzed Logical Plan == event_time: timestamp, event_type: string, product_id: int, category_id: bigint, category_code: string, brand: string, price: double, user_id: int, user_session: string, ingestion_ts: timestamp, event_date: date, price_tier: string Project [event_time#13574, event_type#13575, product_id#13576, category_id#13577L, category_code#13578, brand#13579, price#13580, user_id#13581, user_session#13582, ingestion_ts#13583, event_date#13584, price_tier#13585] +- SubqueryAlias workspace.silver.ecommerce_cleaned +- Relation workspace.silver.ecommerce_cleaned[event_time#13574,event_type#13575,product_id#13576,category_id#13577L,category_code#13578,brand#13579,price#13580,user_id#13581,user_session#13582,ingestion_ts#13583,event_date#13584,price_tier#13585] parquet == Optimized Logical Plan == Relation workspace.silver.ecommerce_cleaned[event_time#13574,event_type#13575,product_id#13576,category_id#13577L,category_code#13578,brand#13579,price#13580,user_id#13581,user_session#13582,ingestion_ts#13583,event_date#13584,price_tier#13585] parquet == Physical Plan == *(1) ColumnarToRow +- PhotonResultStage +- PhotonScan parquet workspace.silver.ecommerce_cleaned[event_time#13574,event_type#13575,product_id#13576,category_id#13577L,category_code#13578,brand#13579,price#13580,user_id#13581,user_session#13582,ingestion_ts#13583,event_date#13584,price_tier#13585] DataFilters: [], DictionaryFilters: [], Format: parquet, Location: PreparedDeltaFileIndex(1 paths)[s3://dbstorage-prod-bmylb/uc/ae6e4cfa-4597-46fb-a4e9-eeb27a5fb90a..., OptionalDataFilters: [], PartitionFilters: [], ReadSchema: struct<event_time:timestamp,event_type:string,product_id:int,category_id:bigint,category_code:str..., RequiredDataFilters: [] == Photon Explanation == The query is fully supported by Photon. == Optimizer Statistics (table names per statistics state) == missing = partial = full = ecommerce_cleaned"


----

#### ***2. Partition large tables***

In [0]:
spark.sql("""
  CREATE TABLE silver.events_part
  USING DELTA
  PARTITIONED BY (event_date, event_type)
  AS SELECT * FROM workspace.silver.ecommerce_cleaned
""")

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

---

#### ***3. Apply ZORDER***

In [0]:
spark.sql("OPTIMIZE silver.events_part ZORDER BY (user_id, product_id)")

DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,clusteringStats:struct<inputZCubeFiles:struct<numFiles:bigint,size:bigint>,inputOtherFiles:struct<numFiles:bigint,size:bigint>,inputNumZCubes:bigint,mergedFiles:struct<numFiles:bigint,size:bigint>,numOutputZCubes:bigint>,numBins:bigint,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,

---

#### ***4. Benchmark improvements***

In [0]:
# Benchmark
import time
start = time.time()
spark.sql("SELECT * FROM workspace.silver.ecommerce_cleaned WHERE user_id=12345").count()
print(f"Time: {time.time()-start:.2f}s")

Time: 0.76s


### **Resources**
- [Performance Tuning](https://docs.databricks.com/performance/)
- [Optimization guide](https://docs.databricks.com/delta/optimizations-oss.html)

----